# Data Wrangling for datasets | ADSE 2025

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import warnings
sns.set(style='whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

### Correlate Anemia Prevalence, Poverty level, and Malnutrition prevalence

In [26]:
Anemia_U5 = pd.read_csv(r'C:\Users\Leon Pascual\Documents\GitHub\CPE311_Computational-Thinking-with-Python\Final\ADSE\datasets\API_SH.ANM.CHLD.ZS_DS2_en_csv_v2_104143\PVA_U5.csv', skiprows=4)
Anemia_U5.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2016,2017,2018,2019,2020,2021,2022,2023,2024,Unnamed: 69
0,Aruba,ABW,Prevalence of anemia among children (% of chil...,SH.ANM.CHLD.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Africa Eastern and Southern,AFE,Prevalence of anemia among children (% of chil...,SH.ANM.CHLD.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,54.172067,54.293680,54.361880,54.445964,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,AFG,Prevalence of anemia among children (% of chil...,SH.ANM.CHLD.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,43.900000,44.200000,44.500000,44.900000,NaN,NaN,NaN,NaN,NaN,NaN
3,Africa Western and Central,AFW,Prevalence of anemia among children (% of chil...,SH.ANM.CHLD.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,71.500043,70.826525,69.992481,69.070796,NaN,NaN,NaN,NaN,NaN,NaN
4,Angola,AGO,Prevalence of anemia among children (% of chil...,SH.ANM.CHLD.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,61.900000,62.100000,62.300000,62.400000,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
# drop empty columns
Anemia_U5 = Anemia_U5.drop(['1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968',
       '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977',
       '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986',
       '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995',
       '1996', '1997', '1998', '1999'], axis=1)

In [31]:
print(Anemia_U5['Country Name'].unique())


['Aruba' 'Africa Eastern and Southern' 'Afghanistan'
 'Africa Western and Central' 'Angola' 'Albania' 'Andorra' 'Arab World'
 'United Arab Emirates' 'Argentina' 'Armenia' 'American Samoa'
 'Antigua and Barbuda' 'Australia' 'Austria' 'Azerbaijan' 'Burundi'
 'Belgium' 'Benin' 'Burkina Faso' 'Bangladesh' 'Bulgaria' 'Bahrain'
 'Bahamas, The' 'Bosnia and Herzegovina' 'Belarus' 'Belize' 'Bermuda'
 'Bolivia' 'Brazil' 'Barbados' 'Brunei Darussalam' 'Bhutan' 'Botswana'
 'Central African Republic' 'Canada' 'Central Europe and the Baltics'
 'Switzerland' 'Channel Islands' 'Chile' 'China' "Cote d'Ivoire"
 'Cameroon' 'Congo, Dem. Rep.' 'Congo, Rep.' 'Colombia' 'Comoros'
 'Cabo Verde' 'Costa Rica' 'Caribbean small states' 'Cuba' 'Curacao'
 'Cayman Islands' 'Cyprus' 'Czechia' 'Germany' 'Djibouti' 'Dominica'
 'Denmark' 'Dominican Republic' 'Algeria'
 'East Asia & Pacific (excluding high income)'
 'Early-demographic dividend' 'East Asia & Pacific'
 'Europe & Central Asia (excluding high income)' 'Europ

In [38]:
southeast_asia = [
    "Brunei Darussalam",
    "Cambodia",
    "Indonesia",
    "Lao PDR",
    "Malaysia",
    "Myanmar",
    "Philippines",
    "Singapore",
    "Thailand",
    "Timor-Leste",
    "Viet Nam"
]
SEA_Anemia_U5 = Anemia_U5.query("`Country Name` in @southeast_asia")
SEA_Anemia_U5.head(15)

,Country Name,Country Code,Indicator Name,Indicator Code,2000,2001,2002,2003,2004,2005,...,2016,2017,2018,2019,2020,2021,2022,2023,2024,Unnamed: 69
31,Brunei Darussalam,BRN,Prevalence of anemia among children (% of chil...,SH.ANM.CHLD.ZS,17.0,16.7,16.6,16.7,16.6,16.7,...,18.2,18.8,19.3,20.0,NaN,NaN,NaN,NaN,NaN,NaN
106,Indonesia,IDN,Prevalence of anemia among children (% of chil...,SH.ANM.CHLD.ZS,43.9,43.0,42.1,41.2,40.5,39.8,...,37.8,38.0,38.2,38.4,NaN,NaN,NaN,NaN,NaN,NaN
123,Cambodia,KHM,Prevalence of anemia among children (% of chil...,SH.ANM.CHLD.ZS,65.9,64.8,63.6,62.4,61.3,60.3,...,50.9,50.3,49.6,49.0,NaN,NaN,NaN,NaN,NaN,NaN
129,Lao PDR,LAO,Prevalence of anemia among children (% of chil...,SH.ANM.CHLD.ZS,50.2,49.3,48.4,47.7,46.9,46.2,...,41.4,41.3,41.3,41.4,NaN,NaN,NaN,NaN,NaN,NaN
160,Myanmar,MMR,Prevalence of anemia among children (% of chil...,SH.ANM.CHLD.ZS,57.5,56.6,55.8,55.0,54.2,53.5,...,49.6,49.6,49.6,49.6,NaN,NaN,NaN,NaN,NaN,NaN
169,Malaysia,MYS,Prevalence of anemia among children (% of chil...,SH.ANM.CHLD.ZS,20.2,19.8,19.6,19.4,19.4,19.5,...,22.5,23.1,23.8,24.6,NaN,NaN,NaN,NaN,NaN,NaN
187,Philippines,PHL,Prevalence of anemia among children (% of chil...,SH.ANM.CHLD.ZS,35.0,34.0,32.9,31.8,30.6,29.2,...,14.7,14.2,13.8,13.5,NaN,NaN,NaN,NaN,NaN,NaN
208,Singapore,SGP,Prevalence of anemia among children (% of chil...,SH.ANM.CHLD.ZS,11.4,11.2,11.1,11.1,11.1,11.1,...,12.6,12.9,13.2,13.5,NaN,NaN,NaN,NaN,NaN,NaN
233,Thailand,THA,Prevalence of anemia among children (% of chil...,SH.ANM.CHLD.ZS,26.5,26.2,26.0,25.7,25.4,25.2,...,24.5,24.6,24.7,24.9,NaN,NaN,NaN,NaN,NaN,NaN
237,Timor-Leste,TLS,Prevalence of anemia among children (% of chil...,SH.ANM.CHLD.ZS,51.3,49.9,48.7,47.7,46.8,46.3,...,46.0,46.1,46.1,46.3,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
GINI = pd.read_csv(r'C:\Users\Leon Pascual\Documents\GitHub\CPE311_Computational-Thinking-with-Python\Final\ADSE\datasets\API_SI.POV.GINI_DS2_en_csv_v2_131994\GINI Index.csv', skiprows=4)
GINI.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2016,2017,2018,2019,2020,2021,2022,2023,2024,Unnamed: 69
0,Aruba,ABW,Gini index,SI.POV.GINI,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Africa Eastern and Southern,AFE,Gini index,SI.POV.GINI,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,AFG,Gini index,SI.POV.GINI,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Africa Western and Central,AFW,Gini index,SI.POV.GINI,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Angola,AGO,Gini index,SI.POV.GINI,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,51.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
